In [ ]:
import sys 
import sqlite3
import argparse
import os 
from dotenv import load_dotenv
from openai import OpenAI
load_dotenv()

In [ ]:
# We need to create a connection to the database file, named vocab.db
# We have retrieved this file from our kindle, and it contains the words we have looked up

def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except sqlite3.Error as e:
        print(e)
    return conn

In [ ]:
# We want to joint the two tables from LOOKUPS and WORDS
# We can do this by making use of word_key from LOOKUPS and id from WORDS
# The only columns we need are: word_key/id, word, stem, lang, and usage
def join_tables(conn):
    cur = conn.cursor()
    cur.execute("SELECT LOOKUPS.word_key, WORDS.word, WORDS.stem, WORDS.lang, LOOKUPS.usage FROM LOOKUPS INNER JOIN WORDS ON LOOKUPS.word_key = WORDS.id")
    rows = cur.fetchall()
    return rows

In [ ]:
# Let's try our code
def main():
    conn = create_connection("vocab.db")
    with conn:
        rows = join_tables(conn)
        return rows

In [ ]:
main()

In [ ]:
# Let's create an OpenAI client to use the API
client = OpenAI()

In [ ]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "Write a haiku about recursion in programming."
        }
    ]
)

print(completion.choices[0].message)

In [ ]:
# For each word in our joined tables, we want to determine the meaning of the word in the context of the usage. 
# We will use the OpenAI client to generate a response for each word
def generate_definition(word, usage):
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "content": f"Define \"{word}\" in the context of \"{usage}\"."
            }
        ]
    )
    return completion.choices[0].message

In [ ]:
# Take first 10 rows for testing
rows = main()
rows = rows[:10]
responses = []
for row in rows:
    word = row[1]
    usage = row[4]
    response = generate_definition(word, usage)
    responses.append(response)


In [ ]:
responses